In [ ]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import time
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
data_path='drive/MyDrive/Project 1: Face Recognition/Datasets'
image_folder_path=os.path.join(data_path, "Images")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.listdir(data_path)

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'gender_classification.csv'))
df.columns=["image_path","Male"]
df

In [ ]:
#data process male or female
df_male = df.loc[df['Male']==1]
df_female = df.loc[df['Male']==0]

# **Set Data Train and Test**

In [ ]:
def SplitDataset(split_ratio):
    # Split the data male into train and test sets
    split_index_m = int(len(data_male) * split_ratio)
    train_data_m = data_male[:split_index_m]
    test_data_m = data_male[split_index_m:]


    # Split the data female into train and test sets
    split_index = int(len(data_female) * split_ratio)
    train_data_f = data_female[:split_index]
    test_data_f = data_female[split_index:]


    train_data = pd.concat([train_data_m,train_data_f])
    test_data = pd.concat([test_data_m,test_data_f])

    return train_data, test_data

In [ ]:
# Shuffle the dataframe to ensure randomness
data_male = df_male.sample(frac=1)
data_female = df_female.sample(frac=1)

# Define the train-test split ratio
split_ratio = 0.9

# Split the data male into train and test sets
#split_index_m = int(len(data_male) * split_ratio)
#train_data_m = data_male[:split_index_m]
#test_data_m = data_male[split_index_m:]


# Split the data female into train and test sets
#split_index = int(len(data_female) * split_ratio)
#train_data_f = data_female[:split_index]
#test_data_f = data_female[split_index:]


#train_data = pd.concat([train_data_m,train_data_f])
#test_data = pd.concat([test_data_m,test_data_f])

train_data, test_data = SplitDataset(split_ratio)

In [ ]:
split_ratio = 0.8
train_data, test_data = SplitDataset(split_ratio)

In [ ]:
split_ratio = 0.7
train_data, test_data = SplitDataset(split_ratio)

In [ ]:
class GenderDataset(Dataset):
    def __init__(self, data, image_folder_path, transform=None):
        self.data = data
        self.image_folder_path = image_folder_path
        self.transform = transform
        self.classes = ['Female', 'Male']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_folder_path, self.data.iloc[idx, 0])
        image = Image.open(image_path).convert('RGB')
        gender = self.data.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(gender, dtype=torch.long)

In [ ]:
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



# Create the dataloaders for the train and test datasets
train_dataset = GenderDataset(train_data, image_folder_path, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=2)

test_dataset = GenderDataset(test_data, image_folder_path, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True,num_workers=2)
class_names = train_dataset.classes

train_dataset[0], test_dataset[0], class_names

In [ ]:
#show an image
def imgshow(inp, title=None):
  inp = inp.numpy().transpose((1, 2, 0))
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)
  plt.imshow(inp)
  if title is not None:
    plt.title(title)
  plt.pause(0.001)

# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imgshow(out, title=[class_names[x] for x in classes])

In [ ]:
# Define the GoogLeNet model and modify it to have 2 output classes
model_ft = models.googlenet(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

# Define the learning rate to be used by the optimizer
LEARNING_RATE = 0.001

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)

# **Build** **Model**

In [ ]:
!pip install torcheval

In [ ]:
from torcheval.metrics.functional import multiclass_f1_score

def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, use_gpu=torch.cuda.is_available(), num_epochs=10):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    test_preds = torch.tensor([], dtype=torch.long).to("cuda")
    test_labels = torch.tensor([], dtype=torch.long).to("cuda")

    # eval
    epoch_loss_list=[]
    epoch_acc_list=[]
    f1_score_list=[]
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                #scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                

                # backward + optimize only if in training phase
                if phase == 'train':
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                else:
                    _, preds = torch.max(outputs.data, 1)
                    loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

                test_preds = torch.cat((test_preds, preds), dim=0)
                test_labels = torch.cat((test_labels, labels.data), dim=0)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.float() / dataset_sizes[phase]
            f1_score = multiclass_f1_score(test_preds, test_labels, num_classes=2, average='micro')

            # save eval score
            epoch_loss_list+=[epoch_loss]
            epoch_acc_list+=[epoch_acc]
            f1_score_list+=[f1_score]

            print('{} Loss: {:.4f} Acc: {:.4f} f1_score {:.4f}'.format(
                phase, epoch_loss, epoch_acc, f1_score))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                state = {'model':model_ft.state_dict(),'optim':optimizer_ft.state_dict()}
                torch.save(state,'/content/drive/MyDrive/save model/copy_point_googlenet_90_10_best.pth')
                print('saving model')

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_loss_list, epoch_acc_list, f1_score_list

In [ ]:
dataloders = {
    "train":train_loader, "test":test_loader
}
dataset_sizes= {
    "train":len(train_dataset), "test":len(test_dataset)
}

In [ ]:
use_gpu = torch.cuda.is_available()

if use_gpu:
  model_ft = model_ft.to("cuda")

In [ ]:
#define epochs
NUM_EPOCHS = 10
model_ft, epoch_loss_list, epoch_acc_list, f1_score_list = train_model(model_ft, dataloders, dataset_sizes, criterion, optimizer_ft, use_gpu, NUM_EPOCHS)

# **Evaluation**

In [ ]:
from sklearn.metrics import confusion_matrix

def evaluate_model(model, test_loader, target_labels):
    checkpoint = torch.load('/content/drive/MyDrive/save model/copy_point_googlenet_90_10_best.pth')
    model.load_state_dict(checkpoint['model'])

    model.eval()
    test_preds = torch.tensor([], dtype=torch.long).to(device)
    test_labels = torch.tensor([], dtype=torch.long).to(device)
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            # wrap them in Variable
            if use_gpu:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            test_preds = torch.cat((test_preds, predicted), dim=0)
            test_labels = torch.cat((test_labels, labels), dim=0)

    accuracy = 100 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        accuracy))
    

    return test_preds, test_labels

In [ ]:
target_labels=class_names
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_pred, test_label = evaluate_model(model_ft, test_loader, target_labels)

In [ ]:
# Plot the confusion matrix
test_pred , test_label

cm = confusion_matrix(test_label.cpu().numpy(), test_pred.cpu().numpy())
fig, ax = plt.subplots(figsize=(4, 4))
ax.imshow(cm)
ax.grid(False)
ax.set_xlabel('Predicted labels', fontsize=12, color='black')
ax.set_ylabel('True labels', fontsize=12, color='black')
ax.set_xticks(range(len(target_labels)))
ax.set_yticks(range(len(target_labels)))
ax.set_xticklabels(target_labels, fontsize=12, rotation=90)
ax.set_yticklabels(target_labels, fontsize=12)
for i in range(len(target_labels)):
  for j in range(len(target_labels)):
      ax.text(j, i, format(cm[i, j], 'd'), ha="center", va="center", color="white" if cm[i, j] > (cm.max() / 2.) else "black", fontsize=12)
plt.show()

# **PREDICT**

In [ ]:
test_loaders = DataLoader(test_dataset, batch_size=2, shuffle=True,num_workers=2)
checkpoint = torch.load('/content/drive/MyDrive/save model/copy_point_googlenet_90_10_best.pth')
model_ft.load_state_dict(checkpoint['model'])

# set model to evaluation mode
model_ft.eval()

# get a batch of test data
images, labels = next(iter(test_loaders))

# make predictions
with torch.no_grad():
    images = images.to(device)
    labels = labels.to(device)
    outputs = model_ft(images)
    _, preds = torch.max(outputs, 1)

# display results
for i in range(images.shape[0]):
    image = images[i].cpu().numpy().transpose((1, 2, 0))
    label = class_names[labels[i]]
    pred = class_names[preds[i]]
    
    fig, ax = plt.subplots(figsize=(4, 4))
    plt.imshow(image)
    plt.title(f'True Label: {label}, Predicted Label: {pred}')
    plt.show()